---
title: "They can't hide behind Saros"
description: "Juuse Saros is having a tough start, but it's not like he's getting any help"
author: "chicken"
date: "11/22/2025"
categories:
    - "nashville predators"
image: charts/5v5_xgf_xga_NSH.png
lightbox: true
draft: true
---

## Housekeeping

### Import dependencies and set options

In [ ]:
# chickenstats library and utilities
from chickenstats.chicken_nhl import Scraper, Season
from chickenstats.chicken_nhl._helpers import charts_directory


# plotting library and utilities
import matplotlib.pyplot as plt
import pandas as pd
import polars as pl

# miscellaneous utilities
from dotenv import load_dotenv
import os

In [ ]:
# set pandas options so that there are no columns abbreviated
pd.set_option("display.max_columns", None)
pl.Config.set_tbl_cols(-1)

polars.config.Config

### Create directory for charts

In [ ]:
charts_directory()

### chickenstats matplotlib styles

In [ ]:
plt.style.use("chickenstats")  # this is available when you import chickenstats.utilities

### Getting and setting environment variables

In [ ]:
load_dotenv()
api_user = os.environ.get("API_USER")
api_password = os.environ.get("API_PASSWORD")

## Scraping and prepping data

### Scraping schedule and game IDs

In [ ]:
years = list(range(2015, 2026))

game_ids = []

for year in years:
    season = Season(year, backend="polars")

    teams = "NSH"
    disable_progress_bar = True

    if year == 2025:
        teams = None
        disable_progress_bar = False

    schedule = season.schedule(teams=teams, disable_progress_bar=disable_progress_bar)

    final_games = schedule.filter(pl.col("game_state") == "OFF")

    season_game_ids = final_games["game_id"].to_list()

    game_ids.extend(season_game_ids)

    if year == 2025:
        latest_date = final_games["game_date"].to_list()[-1]

Output()

### Scraping play-by-play data

In [ ]:
scraper = Scraper(game_ids, backend="polars")

In [ ]:
play_by_play = scraper.play_by_play

Output()

### Prepping the stats dataframes

In [ ]:
game_stats = scraper.stats

Output()

In [ ]:
scraper.prep_stats(level="season")
season_stats = scraper.stats

Output()

## Munging the data

### Game stats goals scored and saved above expected

In [ ]:
game_stats = game_stats.with_columns(
    gsax_skater=pl.when(pl.col("position") != "G").then(pl.col("g") - pl.col("ixg")).otherwise(0),
    gsax_goalie=pl.when(pl.col("position") == "G").then(pl.col("xga") - pl.col("ga")).otherwise(0),
)

## Individual stats for line chart

In [ ]:
play_by_play["goalie_gsax"] = play_by_play.pred_goal - play_by_play.goal

In [ ]:
goalie = "JUUSE SAROS"

group_list = [
    "season",
    "session",
    "game_id",
    "game_date",
    "strength_state",
    "opp_team",
    "opp_goalie",
    "opp_goalie_eh_id",
]

shot_events = ["GOAL", "SHOT", "BLOCK", "MISS"]

condition = play_by_play.event.isin(shot_events)

# play_by_play["goalie_cum_shots"] = np.nan

play_by_play["goalie_cum_shots"] = play_by_play.loc[condition].groupby(group_list).event.transform("cumcount") + 1

play_by_play["goalie_cum_gsax"] = play_by_play.loc[condition].groupby(group_list).goalie_gsax.transform("cumsum")

play_by_play.loc[condition]

,id,season,session,game_id,game_date,event_idx,period,period_seconds,game_seconds,strength_state,event_team,opp_team,event,description,zone,coords_x,coords_y,danger,high_danger,player_1,player_1_eh_id,player_1_eh_id_api,player_1_api_id,player_1_position,player_1_type,player_2,player_2_eh_id,player_2_eh_id_api,player_2_api_id,player_2_position,player_2_type,player_3,player_3_eh_id,player_3_eh_id_api,player_3_api_id,player_3_position,player_3_type,score_state,score_diff,forwards_percent,opp_forwards_percent,shot_type,event_length,event_distance,pbp_distance,event_angle,penalty,penalty_length,home_score,home_score_diff,away_score,away_score_diff,is_home,is_away,home_team,away_team,home_skaters,away_skaters,home_on,home_on_eh_id,home_on_api_id,home_on_positions,away_on,away_on_eh_id,away_on_api_id,away_on_positions,event_team_skaters,teammates,teammates_eh_id,teammates_api_id,teammates_positions,own_goalie,own_goalie_eh_id,own_goalie_api_id,forwards,forwards_eh_id,forwards_api_id,forwards_count,defense,defense_eh_id,defense_api_id,defense_count,opp_strength_state,opp_score_state,opp_score_diff,opp_team_skaters,opp_team_on,opp_team_on_eh_id,opp_team_on_api_id,opp_team_on_positions,opp_goalie,opp_goalie_eh_id,opp_goalie_api_id,opp_forwards,opp_forwards_eh_id,opp_forwards_api_id,opp_forwards_count,opp_defense,opp_defense_eh_id,opp_defense_api_id,opp_defense_count,home_forwards,home_forwards_eh_id,home_forwards_api_id,home_forwards_count,home_forwards_percent,home_defense,home_defense_eh_id,home_defense_api_id,home_defense_count,home_goalie,home_goalie_eh_id,home_goalie_api_id,away_forwards,away_forwards_eh_id,away_forwards_api_id,away_forwards_count,away_forwards_percent,away_defense,away_defense_eh_id,away_defense_api_id,away_defense_count,away_goalie,away_goalie_eh_id,away_goalie_api_id,change_on_count,change_off_count,change_on,change_on_eh_id,change_on_api_id,change_on_positions,change_off,change_off_eh_id,change_off_api_id,change_off_positions,change_on_forwards_count,change_off_forwards_count,change_on_forwards,change_on_forwards_eh_id,change_on_forwards_api_id,change_off_forwards,change_off_forwards_eh_id,change_off_forwards_api_id,change_on_defense_count,change_off_defense_count,change_on_defense,change_on_defense_eh_id,change_on_defense_api_id,change_off_defense,change_off_defense_eh_id,change_off_defense_api_id,change_on_goalie_count,change_off_goalie_count,change_on_goalie,change_on_goalie_eh_id,change_on_goalie_api_id,change_off_goalie,change_off_goalie_eh_id,change_off_goalie_api_id,pred_goal,pred_goal_adj,goal,goal_adj,hd_goal,shot,shot_adj,hd_shot,miss,miss_adj,hd_miss,fenwick,fenwick_adj,hd_fenwick,corsi,corsi_adj,block,block_adj,teammate_block,teammate_block_adj,hit,give,take,fac,penl,change,stop,chl,ozf,nzf,dzf,ozc,nzc,dzc,otf,pen0,pen2,pen4,pen5,pen10,goalie_gsax,goalie_cum_shots,goalie_cum_gsax
5,20150200090006,20152016,R,2015020009,2015-10-08,6,1,13,13,5v5,NSH,CAR,SHOT,"NSH ONGOAL - #59 JOSI, SNAP, OFF. ZONE, 63 FT.",OFF,32.0,27.0,0.0,0.0,ROMAN JOSI,ROMAN.JOSI,ROMAN.JOSI,8474600,D,SHOOTER,None,None,None,NaN,None,None,None,None,None,NaN,None,None,0v0,0,0.600000,0.600000,SNAP,5,63.071388,63.0,25.346176,None,NaN,0,0,0,0,1,0,NSH,CAR,5,5,"FILIP FORSBERG, JAMES NEAL, MIKE RIBEIRO, SHEA...","FILIP.FORSBERG, JAMES.NEAL, MIKE.RIBEIRO, SHEA...","8476887, 8471707, 8467371, 8470642, 8474600","L, L, C, D, D","ERIC STAAL, ELIAS LINDHOLM, KRIS VERSTEEG, JUS...","ERIC.STAAL, ELIAS.LINDHOLM, KRIS.VERSTEEG, JUS...","8470595, 8477496, 8471346, 8475753, 8468493","C, C, R, D, D",5,"FILIP FORSBERG, JAMES NEAL, MIKE RIBEIRO, SHEA...","FILIP.FORSBERG, JAMES.NEAL, MIKE.RIBEIRO, SHEA...","8476887, 8471707, 8467371, 8470642, 8474600","L, L, C, D, D",PEKKA RINNE,PEKKA.RINNE,8471469,"FILIP FORSBERG, JAMES NEAL, MIKE RIBEIRO","FILIP.FORSBERG, JAMES.NEAL, MIKE.RIBEIRO","8476887, 8471707, 8467371",3,"SHEA WEBER, ROMAN JOSI","SHEA.WEBER, ROMAN.JOSI","8470642, 8474600",2,5v5,0v0,0,5,"ERIC STAAL, ELIAS LINDHOLM, KRIS VERSTEEG, JUS...","ERIC.STAAL,

In [ ]:
game_stats

,season,session,game_id,game_date,player,eh_id,api_id,position,team,opp_team,strength_state,toi,g,g_adj,ihdg,a1,a2,ixg,ixg_adj,isf,isf_adj,ihdsf,imsf,imsf_adj,ihdm,iff,iff_adj,ihdf,isb,isb_adj,icf,icf_adj,ibs,ibs_adj,igive,itake,ihf,iht,ifow,ifol,iozfw,iozfl,inzfw,inzfl,idzfw,idzfl,a1_xg,a2_xg,ipent0,ipent2,ipent4,ipent5,ipent10,ipend0,ipend2,ipend4,ipend5,ipend10,gf,ga,gf_adj,ga_adj,hdgf,hdga,xgf,xga,xgf_adj,xga_adj,sf,sa,sf_adj,sa_adj,hdsf,hdsa,ff,fa,ff_adj,fa_adj,hdff,hdfa,cf,ca,cf_adj,ca_adj,bsf,bsa,bsf_adj,bsa_adj,msf,msa,msf_adj,msa_adj,hdmsf,hdmsa,teammate_block,teammate_block_adj,hf,ht,give,take,ozf,nzf,dzf,fow,fol,ozfw,ozfl,nzfw,nzfl,dzfw,dzfl,pent0,pent2,pent4,pent5,pent10,pend0,pend2,pend4,pend5,pend10,ozs,nzs,dzs,otf,g_p60,g_adj_p60,ihdg_p60,a1_p60,a2_p60,ixg_p60,ixg_adj_p60,isf_p60,isf_adj_p60,ihdsf_p60,imsf_p60,imsf_adj_p60,ihdm_p60,iff_p60,iff_adj_p60,ihdf_p60,isb_p60,isb_adj_p60,icf_p60,icf_adj_p60,ibs_p60,ibs_adj_p60,igive_p60,itake_p60,ihf_p60,iht_p60,a1_xg_p60,a2_xg_p60,ipent0_p60,ipent2_p60,ipent4_p60,ipent5_p60,ipent10_p60,ipend0_p60,ipend2_p60,ipend4_p60,ipend5_p60,ipend10_p60,gf_p60,ga_p60,gf_adj_p60,ga_adj_p60,hdgf_p60,hdga_p60,xgf_p60,xga_p60,xgf_adj_p60,xga_adj_p60,sf_p60,sa_p60,sf_adj_p60,sa_adj_p60,hdsf_p60,hdsa_p60,ff_p60,fa_p60,ff_adj_p60,fa_adj_p60,hdff_p60,hdfa_p60,cf_p60,ca_p60,cf_adj_p60,ca_adj_p60,bsf_p60,bsa_p60,bsf_adj_p60,bsa_adj_p60,msf_p60,msa_p60,msf_adj_p60,msa_adj_p60,hdmsf_p60,hdmsa_p60,teammate_block_p60,teammate_block_adj_p60,hf_p60,ht_p60,give_p60,take_p60,pent0_p60,pent2_p60,pent4_p60,pent5_p60,pent10_p60,pend0_p60,pend2_p60,pend4_p60,pend5_p60,pend10_p60,gf_percent,gf_adj_percent,hdgf_percent,xgf_percent,xgf_adj_percent,sf_percent,sf_adj_percent,hdsf_percent,ff_percent,ff_adj_percent,hdff_percent,cf_percent,cf_adj_percent,bsf_percent,bsf_adj_percent,msf_percent,msf_adj_percent,hdmsf_percent,hf_percent,take_percent,gsax_skater,gsax_goalie
0,20152016,P,2015030171,2016-04-15,ANDREW COGLIANO,ANDREW.COGLIANO,8471699,C,ANA,NSH,4v5,1.250000,0,0.0,0,0,0,0.000000,0.000000,0,0.000000,0,0,0.000000,0,0,0.000000,0,0,0.0,0,0.000000,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.144220,0.000000,0.143804,0,1,0.000000,0.949764,0,0,0,2,0.000000,1.918223,0,0,0,2,0.000000,1.918223,0,0,0.0,0.0,0,1,0.000000,0.959111,0,0,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.922569,0.000000,6.902576,0.000000,48.000000,0.000000,45.588661,0.000000,0.000000,0.000000,96.000000,0.000000,92.074691,0.000000,0.000000,0.000000,96.000000,0.000000,92.074691,0.000000,0.000000,0.0,0.0,0.000000,48.000000,0.000000,46.037346,0.000000,0.00000,0.0,0.0,48.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.0,1.000000,0.0,0.000000,0.0
1,20152016,P,2015030171,2016-04-15,ANDREW COGLIANO,ANDREW.COGLIANO,8471699,C,ANA,NSH,5v4,0.083333,0,0.0,0,0,0,0.000000,0.000000,0,0.000000,0,0,0.000000,0,0,0.000000,0,0,0.0,0,0.000000,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0,0,0.0,0.0,0,0,0.000000,0.000000,0,0,0,0.0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00

In [ ]:
season_stats

,season,session,player,eh_id,api_id,position,team,strength_state,toi,g,g_adj,ihdg,a1,a2,ixg,ixg_adj,isf,isf_adj,ihdsf,imsf,imsf_adj,ihdm,iff,iff_adj,ihdf,isb,isb_adj,icf,icf_adj,ibs,ibs_adj,igive,itake,ihf,iht,ifow,ifol,iozfw,iozfl,inzfw,inzfl,idzfw,idzfl,a1_xg,a2_xg,ipent0,ipent2,ipent4,ipent5,ipent10,ipend0,ipend2,ipend4,ipend5,ipend10,gf,ga,gf_adj,ga_adj,hdgf,hdga,xgf,xga,xgf_adj,xga_adj,sf,sa,sf_adj,sa_adj,hdsf,hdsa,ff,fa,ff_adj,fa_adj,hdff,hdfa,cf,ca,cf_adj,ca_adj,bsf,bsa,bsf_adj,bsa_adj,msf,msa,msf_adj,msa_adj,hdmsf,hdmsa,teammate_block,teammate_block_adj,hf,ht,give,take,ozf,nzf,dzf,fow,fol,ozfw,ozfl,nzfw,nzfl,dzfw,dzfl,pent0,pent2,pent4,pent5,pent10,pend0,pend2,pend4,pend5,pend10,ozs,nzs,dzs,otf,g_p60,g_adj_p60,ihdg_p60,a1_p60,a2_p60,ixg_p60,ixg_adj_p60,isf_p60,isf_adj_p60,ihdsf_p60,imsf_p60,imsf_adj_p60,ihdm_p60,iff_p60,iff_adj_p60,ihdf_p60,isb_p60,isb_adj_p60,icf_p60,icf_adj_p60,ibs_p60,ibs_adj_p60,igive_p60,itake_p60,ihf_p60,iht_p60,a1_xg_p60,a2_xg_p60,ipent0_p60,ipent2_p60,ipent4_p60,ipent5_p60,ipent10_p60,ipend0_p60,ipend2_p60,ipend4_p60,ipend5_p60,ipend10_p60,gf_p60,ga_p60,gf_adj_p60,ga_adj_p60,hdgf_p60,hdga_p60,xgf_p60,xga_p60,xgf_adj_p60,xga_adj_p60,sf_p60,sa_p60,sf_adj_p60,sa_adj_p60,hdsf_p60,hdsa_p60,ff_p60,fa_p60,ff_adj_p60,fa_adj_p60,hdff_p60,hdfa_p60,cf_p60,ca_p60,cf_adj_p60,ca_adj_p60,bsf_p60,bsa_p60,bsf_adj_p60,bsa_adj_p60,msf_p60,msa_p60,msf_adj_p60,msa_adj_p60,hdmsf_p60,hdmsa_p60,teammate_block_p60,teammate_block_adj_p60,hf_p60,ht_p60,give_p60,take_p60,pent0_p60,pent2_p60,pent4_p60,pent5_p60,pent10_p60,pend0_p60,pend2_p60,pend4_p60,pend5_p60,pend10_p60,gf_percent,gf_adj_percent,hdgf_percent,xgf_percent,xgf_adj_percent,sf_percent,sf_adj_percent,hdsf_percent,ff_percent,ff_adj_percent,hdff_percent,cf_percent,cf_adj_percent,bsf_percent,bsf_adj_percent,msf_percent,msf_adj_percent,hdmsf_percent,hf_percent,take_percent
0,20152016,P,ANDREW COGLIANO,ANDREW.COGLIANO,8471699,C,ANA,4v4,1.950000,0,0.000000,0,0,0,0.000000,0.000000,0,0.000000,0,0,0.000000,0,0,0.000000,0,0,0.0,0,0.000000,0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.176592,0.097218,0.178543,0.101526,1,0,1.025229,0.000000,0,0,3,1,3.023736,1.022746,0,1,3,1,3.023736,1.022746,0,0,0.0,0.0,2,1,2.000990,1.022746,0,1,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,30.769231,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.433610,2.991315,5.493645,3.123864,30.769231,0.000000,31.545519,0.000000,0.000000,0.000000,92.307692,30.769231,93.038023,31.469108,0.000000,30.769231,92.307692,30.769231,93.038023,31.469108,0.000000,0.000000,0.0,0.0,61.538462,30.769231,61.568915,31.469108,0.000000,30.769231,0.000000,0.0,0.000000,30.769231,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.644945,0.637498,1.000000,1.000000,0.000000,0.750000,0.747251,0.000000,0.750000,0.747251,0.000000,0.0,0.666667,0.661761,0.000000,0.000000,0.0
1,20152016,P,ANDREW COGLIANO,ANDREW.COGLIANO,8471699,C,ANA,4v5,14.000000,0,0.000000,0,0,0,0.000000,0.000000,0,0.000000,0,0,0.000000,0,0,0.000000,0,0,0.0,0,0.000000,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0.037319,0.608304,0.034143,0.591122,0,7,0.000000,6.433998,0,0,1,10,1.087149,9.559973,0,0,1,11,1.087149,9.559973,0,1,0.0,0.0,1,3,0.925786,3.124025,0,0,0,0.0,3,1,0,0,0,2,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,